# Desafio 1

Conforme definido em sala, os alunos deverão realizar o seguinte desafio pontuado:

- Escolher uma base de dados na área de saúde, educação, segurança pública ou gestão. OBS: o tema dos alunos que faltaram é GESTÃO;
- Criar e enviar o modelo conceitual de Banco de dados;
- Criar o modelo lógico e físico de Banco de Dados;
- Criar o Schema e persistir no Banco de Dados;
- Escolher o dataset, o mesmo deve ser tratado e poderá ter em torno de 1000 linhas (você pode abrir no excel fazer a exclusão de linhas e colunas para ele ficar menor);
- Persistir os dados no banco de dados;
- Fazer todo o processo de CRUD utilizando python;
- Gerar 3 gráficos relacionados ao dataset.
Você deve enviar o dataset tratado, o modelo conceitual, o schema do banco, o arquivo python.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect, text

In [8]:
# Dados do banco

user = "ufrkpl872bmu15"
password = "pfc2f800f1a232ad1055c52d4cbc53018d57166e9190b5215ce20c656fd97ed3b"
host = "cbec45869p4jbu.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com"
port = "5432"
database = "d487kabe2ju107"

# String de conexão
DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Criação do engine
engine = create_engine(DATABASE_URL)

# Testar conexão (opcional)
with engine.connect() as connection:
    result = connection.execute(text("SELECT 1"))  # <-- corrigido aqui
    print(result.scalar())

1
